# Exercício de Análise de Dados 
*** 
<br>

### Cálculo da soma de remunerações em um dataframe a partir de uma lista importada de arquivos PDF.
<br>

<br>

### Importação dos Módulos

<br>

In [1]:
#utilizei o camelot pela sintaxe simples e eficiência
import os
import numpy as np
import pandas as pd
import camelot
import zipfile

<br>

### Importação dos Arquivos PDF e Concatenação da Listagem
<br>
Obs. O Camelot possui um bug: a seleção de várias páginas, (ex. pages='1-3' ou 'all'), não funciona. Então foi necessaŕio repetir a rotina de extração para cada página.

<br>

Isso também fez com que a rotina não funcionasse para o último arquivo, que só possui uma página, gerando erro de índice e obrigando a tratar esse último arquivo separadamente. 
<br>
<br>

In [2]:
List_Path = './Lista_Completa/Arquivos/'
Ult_Path = './Lista_Completa/ult/'
df3 = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8])
df3a = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8])

# Leitura dos arquivos de 3 páginas
for arq in os.listdir(List_Path):
    df1 = camelot.read_pdf(List_Path+arq, pages='1') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    df1 = camelot.read_pdf(List_Path+arq, pages='2') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    df1 = camelot.read_pdf(List_Path+arq, pages='3') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    
#Leitura do último arquivo com uma só página
for arq in os.listdir(Ult_Path):
    df1a = camelot.read_pdf(Ult_Path+arq, pages='1')
    df2a = df1a[0].df
    df3a = df3a.append(df2a)
df3 = df3.append(df3a)
df3.rename(columns=df3.iloc[0]).drop(df3.index[0])
df3.columns = ['A', 'B', 'C', 'NOME', 'E', 'F', 'G', 'H', 'I']
df3.drop(df3[['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']], axis=1, inplace=True)
df3.drop(df3.index[0], axis=0, inplace=True)

In [12]:
#Criação da Lista Completa
Lista_Completa = pd.DataFrame(columns=["NOME"])
Lista_Completa = Lista_Completa.append(df3)
Lista_Completa = Lista_Completa.sort_values(['NOME']) # Ordenando para conferir a integridade
Lista_Completa = Lista_Completa["NOME"].str.replace('\n', ' ')
Lista_Completa = Lista_Completa.to_frame()
Lista_Completa

,NOME
1,ABDO HASSEN SALMAN
2,ADAILTON ABREU DA SILVA
3,ADALTON FARIA
4,ADEMILDES FERREIRA DE LIMA
5,ADEMIR CARNEIRO DA CUNHA
...,...
5,YURI SILVA SERAFIM MACHADO
6,ZENAR MARIA RIBEIRO MENDES DE SABOIA
7,ZILMA VIEIRA DA CRUZ
8,ZILOCI FERREIRA DOS SANTOS


<br>

### Explorando o Arquivo Geral zipado

<br>

In [5]:
with zipfile.ZipFile('Servidores.zip') as BigFile:
    print(BigFile.namelist(),sep='\n')

['Servidores/', 'Servidores/202001_Remuneracao.csv']


<br>

### Preparação do Arquivo selecionado dentro do zip como DataFrame

<br>

In [6]:
with zipfile.ZipFile('Servidores.zip') as BigFile:
    with BigFile.open('Servidores/202001_Remuneracao.csv') as data:
        dfBig = pd.read_csv(data, sep=';', decimal=",", low_memory=False, encoding='ISO-8859-1') #testar com nrows=10000 primeiro

<br>

### Filtragem dos Dados

<br>

In [11]:
dfBig.columns

Index(['ANO', 'MES', 'Id_SERVIDOR_PORTAL', 'CPF', 'NOME',
       'REMUNERAÇÃO BÁSICA BRUTA (R$)', 'REMUNERAÇÃO BÁSICA BRUTA (U$)',
       'ABATE-TETO (R$)', 'ABATE-TETO (U$)', 'GRATIFICAÇÃO NATALINA (R$)',
       'GRATIFICAÇÃO NATALINA (U$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)', 'FÉRIAS (R$)',
       'FÉRIAS (U$)', 'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)',
       'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)', 'IRRF (R$)', 'IRRF (U$)',
       'PSS/RPGS (R$)', 'PSS/RPGS (U$)', 'DEMAIS DEDUÇÕES (R$)',
       'DEMAIS DEDUÇÕES (U$)', 'PENSÃO MILITAR (R$)', 'PENSÃO MILITAR (U$)',
       'FUNDO DE SAÚDE (R$)', 'FUNDO DE SAÚDE (U$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)',
       'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)',
       'VERBAS INDENIZATÓ

In [24]:
dfWork = dfBig[['NOME','REMUNERAÇÃO BÁSICA BRUTA (R$)']].copy()
dfWork.columns = ['NOME', 'REM']
dfWork


,NOME,REM
0,AARAO CARLOS LUZ MACAMBIRA,7967.90
1,AARAO DIAMANTINO OLIVEIRA,34006.68
2,AARAO FERREIRA LIMA NETO,14249.03
3,AARAO PEREIRA DE ARAUJO JUNIOR,21007.45
4,AARAO SOARES,5896.78
...,...,...
554384,ZUY MARIA MAGRIOTIS,19646.82
554385,ZUZANA RENDAKI MIKOS,4801.59
554386,ZUZITA DOS SANTOS VIANA,6612.02
554387,ZYSMAN NEIMAN,17945.81


In [25]:
#Extração do subconjunto de dados
dfSEL = dfWork.loc[dfWork['NOME'].isin(Lista_Completa['NOME'])]
dfSEL

,NOME,REM
75,ABDO HASSEN SALMAN,19280.76
830,ADAILTON ABREU DA SILVA,17579.95
1320,ADALTON FARIA,20468.03
2484,ADEMILDES FERREIRA DE LIMA,17952.92
2591,ADEMIR CARNEIRO DA CUNHA,21099.50
...,...,...
553120,YURI SILVA SERAFIM MACHADO,20468.03
553639,ZENAR MARIA RIBEIRO MENDES DE SABOIA,31756.08
554034,ZILMA VIEIRA DA CRUZ,21099.50
554070,ZILOCI FERREIRA DOS SANTOS,15536.87


<br>

### Eliminando registros inválidos

<br>

In [28]:
dfSEL.loc[dfSEL["REM"].isnull()].fillna(0)  #verificando valores nulos e substituindo por zero

,NOME,REM


In [31]:
#Transformando em dfloat32 para economizar memória no processamento
dfSEL['REM'].astype(np.float32)

75        19280.759766
830       17579.949219
1320      20468.029297
2484      17952.919922
2591      21099.500000
              ...     
553120    20468.029297
553639    31756.080078
554034    21099.500000
554070    15536.870117
554245    21040.699219
Name: REM, Length: 3601, dtype: float32

<br>

### Soma Final da Remuneração dos Servidores da Lista

<br>

In [33]:
SOMA = dfSEL['REM'].sum()
print("A soma de todas as remunerações da lista é: ")
print(SOMA)

A soma de todas as remunerações da lista é: 
69829588.2
